# Sentiment classification

In [1]:
!pip install perceiver-io==0.2.1

     |████████████████████████████████| 409 kB 7.3 MB/s 
     |████████████████████████████████| 584 kB 37.5 MB/s 
     |████████████████████████████████| 316 kB 50.6 MB/s 
     |████████████████████████████████| 135 kB 33.1 MB/s 
     |████████████████████████████████| 248 kB 34.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 52.4 MB/s 
     |████████████████████████████████| 596 kB 67.3 MB/s 
     |████████████████████████████████| 136 kB 59.9 MB/s 
     |████████████████████████████████| 1.1 MB 45.0 MB/s 
     |████████████████████████████████| 271 kB 56.6 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 57.7 MB/s 
  Created wheel for fairscale: filename=fairscale-0.4.6-py3-none-any.whl size=307252 sha256=dd38be2466a289ff90b29da99af61fc3383541e0249678

In [2]:
# Download tokenizer data
!wget https://raw.githubusercontent.com/krasserm/perceiver-io/main/.cache/imdb-tokenizer-10003.json

--2022-05-09 15:31:11--  https://raw.githubusercontent.com/krasserm/perceiver-io/main/.cache/imdb-tokenizer-10003.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 136806 (134K) [text/plain]
Saving to: ‘imdb-tokenizer-10003.json’

imdb-tokenizer-1000 100%[===================>] 133.60K  --.-KB/s    in 0.02s   

2022-05-09 15:31:12 (5.97 MB/s) - ‘imdb-tokenizer-10003.json’ saved [136806/136806]



In [3]:
# Download model checkpoints
!wget -nc -O logs.zip https://martin-krasser.com/perceiver/logs-update-3.zip
!unzip -qo logs.zip

--2022-05-09 15:31:17--  https://martin-krasser.com/perceiver/logs-update-3.zip
Resolving martin-krasser.com (martin-krasser.com)... 217.160.0.142, 2001:8d8:100f:f000::209
Connecting to martin-krasser.com (martin-krasser.com)|217.160.0.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44295508 (42M) [application/zip]
Saving to: ‘logs.zip’

logs.zip            100%[===================>]  42.24M  20.0MB/s    in 2.1s    

2022-05-09 15:31:20 (20.0 MB/s) - ‘logs.zip’ saved [44295508/44295508]



In [4]:
import torch

from perceiver.data.text_preproc import TextPreprocessor
from perceiver.model.lightning import LitTextClassifier

In [5]:
imdb_preproc = TextPreprocessor(tokenizer_path='imdb-tokenizer-10003.json')

In [6]:
ckpt_path = 'logs/seq_clf/version_1/checkpoints/epoch=035-val_loss=0.259.ckpt'

model = LitTextClassifier.load_from_checkpoint(ckpt_path, clf_ckpt=None).model
model.eval();

In [7]:
text_batch = [
    "I've seen this movie yesterday and it was really boring",
    "I can recommend this movie to all fantasy movie lovers"
]

In [8]:
with torch.no_grad():
    logits = model(*imdb_preproc.preprocess_batch(text_batch))
    preds = logits.argmax(dim=1)

In [9]:
for text, pred in zip(text_batch, preds):
    print(f'{text} (positive = {pred == 1})')

I've seen this movie yesterday and it was really boring (positive = False)
I can recommend this movie to all fantasy movie lovers (positive = True)
